In [14]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import csv
import cv2
import tqdm.notebook as tq
from PIL import Image
import keras.backend as K
import pickle
import keras
from keras.models import load_model
import glob
import os
import ntpath

In [49]:
data_location = 'Dataset/carla_dataset/'
model_location = 'Models/'
noise_location = data_location

In [19]:
model = load_model(model_location + 'model.h5')

with open(model_location + 'mean_image_rgb.pickle', 'rb') as f:
    mean = pickle.load(f, encoding='latin1')  

path_to_weights = model_location + 'yolov3-spp.weights'
path_to_cfg = model_location + 'yolov3-spp.cfg'

network = cv2.dnn.readNetFromDarknet(path_to_cfg, path_to_weights)

network.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
network.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL_FP16)

layers_all = network.getLayerNames()
layers_names_output = [layers_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]
probability_minimum = 0.2
threshold = 0.2

In [20]:
def detect_sign(input_image):
    image_BGR = cv2.imread(input_image)
    h, w = image_BGR.shape[:2]
    blob = cv2.dnn.blobFromImage(image_BGR, 1 / 255.0, (416, 416), swapRB=True, crop=False)

    network.setInput(blob)
    output_from_network = network.forward(layers_names_output)

    bounding_boxes = []
    confidences = []
    for result in output_from_network:
        for detected_objects in result:
            scores = detected_objects[5:]
            class_current = np.argmax(scores)
            confidence_current = scores[class_current]

            if confidence_current > probability_minimum:
                box_current = detected_objects[0:4] * np.array([w, h, w, h])

                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))

                bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))

    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)

    detected_sign = []
    if len(results) > 0:
        for i in results.flatten():
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]
            c_ts = image_BGR[y_min:y_min+int(box_height), x_min:x_min+int(box_width), :]
            if c_ts.shape[:1] == (0,) or c_ts.shape[1:2] == (0,):
                pass
            else:
                blob_ts = cv2.dnn.blobFromImage(c_ts, 1 / 255.0, size=(32, 32), swapRB=True, crop=False)
                blob_ts[0] = blob_ts[0, :, :, :] - mean['mean_image_rgb']
                blob_ts = blob_ts.transpose(0, 2, 3, 1)
                bolb_ts = np.clip(blob_ts[0], 0.0, 1.0)
                
                plt.imsave(input_image.replace('.jpg', '_detect.png'), bolb_ts)
                time.sleep(5)
                X_test = []
                image = Image.open(input_image.replace('.jpg', '_detect.png'))
                image = image.resize((30, 30))
                if image.mode == 'RGBA':
                    image.load() 
                    background = Image.new("RGB", image.size, (255, 255, 255))
                    background.paste(image, mask=image.split()[3])
                    image = np.array(background)
                else:
                    image = np.array(image)
                X_test.append(image)
                X_test = np.array(X_test)
                scores = np.argmax(model.predict(X_test), axis=-1)
                if scores[0] in range(3):
                    box_info = (x_min, y_min, x_min+box_width, y_min+box_height)
                    detected_sign.append((X_test, scores[0], box_info))
                    break
    return detected_sign

In [21]:
class CarSimulatorAdv(tf.keras.Model):

  def __init__(self):
    super(CarSimulatorAdv, self).__init__()
    self.classifier = load_model(model_location+'model.h5')
    with open(model_location+'mean_image_rgb.pickle', 'rb') as f:
        self.mean = pickle.load(f, encoding='latin1')  # dictionary type
    self.objectdetector = cv2.dnn.readNetFromDarknet(model_location+'yolov3-spp.cfg', model_location+'yolov3-spp.weights')
    self.objectdetector.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
    self.objectdetector.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL_FP16)
    self.layers_all = self.objectdetector.getLayerNames()
    self.layers_names_output = [self.layers_all[i[0] - 1] for i in self.objectdetector.getUnconnectedOutLayers()]
    self.probability_minimum = 0.2
    self.threshold = 0.2

  def call(self, input_image, training=False):
    image_BGR = cv2.imread(input_image)
    h, w = image_BGR.shape[:2]

    blob = cv2.dnn.blobFromImage(image_BGR, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    self.objectdetector.setInput(blob)
    output_from_network = self.objectdetector.forward(self.layers_names_output)

    bounding_boxes = []
    confidences = []

    for result in output_from_network:
        for detected_objects in result:
            scores = detected_objects[5:]
            class_current = np.argmax(scores)
            confidence_current = scores[class_current]

            if confidence_current > self.probability_minimum:
                box_current = detected_objects[0:4] * np.array([w, h, w, h])
                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))
                bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))

    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, self.probability_minimum, self.threshold)

    detected_sign = []
    if len(results) > 0:
        for i in results.flatten():
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]
            c_ts = image_BGR[y_min:y_min+int(box_height), x_min:x_min+int(box_width), :]
            if c_ts.shape[:1] == (0,) or c_ts.shape[1:2] == (0,):
                pass
            else:
                blob_ts = cv2.dnn.blobFromImage(c_ts, 1 / 255.0, size=(32, 32), swapRB=True, crop=False)
                blob_ts[0] = blob_ts[0, :, :, :] - self.mean['mean_image_rgb']
                blob_ts = blob_ts.transpose(0, 2, 3, 1)

                if input_image[-4:] == '.png':
                    input_image = input_image.replace('.png', '.jpg')
                tf.keras.preprocessing.image.save_img(input_image.replace('.jpg', '_detect.png'), blob_ts[0], scale=True)
                bolb_img = tf.keras.preprocessing.image.load_img(input_image.replace('.jpg', '_detect.png'), grayscale=False, color_mode='rgb', target_size=(30, 30, 3), interpolation='nearest')
                bolb_ts = keras.preprocessing.image.img_to_array(bolb_img)
                X_test = np.array([bolb_ts])
                scores = np.argmax(self.classifier.predict(X_test), axis=-1)
                #print(self.classifier.predict(X_test))
                if scores[0] in range(3):
                    box_info = (x_min, y_min, x_min+box_width, y_min+box_height)
                    detected_sign.append((X_test, scores[0], box_info))
                    break
    return detected_sign

modelAdv = CarSimulatorAdv()

In [25]:
# set noise class variable to add the noise of that class
noise_class = 2
noise = Image.open(noise_location+'noise'+str(noise_class)+'.png')
if noise.mode == 'RGBA':
    noise.load() 
    background = Image.new("RGB", noise.size, (255, 255, 255))
    background.paste(noise, mask=noise.split()[3])
    noise = np.array(background)
else:
    noise = np.array(noise) 

In [41]:
def generator(sign_image):
    X = Image.open(sign_image)
    X = X.resize((30, 30))
    if X.mode == 'RGBA':
        X.load() 
        background = Image.new("RGB", X.size, (255, 255, 255))
        background.paste(X, mask=X.split()[3])
        X = np.array(background)
    else:
        X = np.array(X)
    X_noise = X + noise
    return X_noise

In [27]:
def image_mapper_adv(input_image, sign, box_info):
    image_path = ntpath.basename(input_image)
    imagebg = Image.open(input_image)
    if imagebg.mode == 'RGBA':
        imagebg.load() 
        background = Image.new("RGB", imagebg.size, (255, 255, 255))
        background.paste(imagebg, mask=imagebg.split()[3])
        imagebg = np.array(background)
    else:
        imagebg = np.array(imagebg)

    x1, y1, x2, y2 = box_info
    X_noisy = Image.open(sign)
    X_noisy = X_noisy.resize((x2-x1, y2-y1))
    if X_noisy.mode == 'RGBA':
        X_noisy.load() 
        background = Image.new("RGB", X_noisy.size, (255, 255, 255))
        background.paste(X_noisy, mask=X_noisy.split()[3])
        X_noisy = np.array(background)
    else:
        X_noisy = np.array(X_noisy)
    
    imagebg[y1:y2, x1:x2] = X_noisy

    tf.keras.preprocessing.image.save_img(input_image.replace(image_path, 'AdvImages/'+image_path.replace('.jpg', '_noisy.png')), imagebg, scale=True)

In [ ]:
with open(data_location + 'carla_dataset_annotation.csv', 'r') as f:
    lines = f.read().splitlines()
    scores = {}
    for l in lines:
        x = l.split(',')
        scores[x[0]] = x[1]

In [51]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
def generate_adv():
    count = 0
    img_count = 0
    index = []
    detect_count = 0
    for image in glob.glob(os.path.join(data_location, '*.jpg')):
        #print(img_count)
        try:
            detect = modelAdv(image)
            if len(detect)>0:
                detect_count+=1
                image_path = ntpath.basename(image)
                if int(scores[image_path]) == detect[-1][1]:
                    sign_img = normalization_layer(detect[-1][0])
                    noisy = generator(image.replace('.jpg', '_detect.png'))
                    noisy_path = data_location + 'NoisySigns/'+image_path.replace('.jpg', '_noisysign.png')
                    tf.keras.preprocessing.image.save_img(noisy_path, noisy, scale=True)
                    image_mapper_adv(image, noisy_path, detect[-1][2])
                    adv_detect = modelAdv(data_location +'AdvImages/'+image_path.replace('.jpg','_noisy.png'))
                    if len(adv_detect)>0:
                        print(image_path, detect[-1][1], adv_detect[-1][1])
                        if int(scores[image_path]) != adv_detect[-1][1]:
                            count+=1
                            index.append(image_path)
                # if img_count == 10:
                #     break
        except:
            continue
        img_count+=1
    print(detect_count)
    return count, index

In [ ]:
count = generate_adv()

In [ ]:
count[0], count[1]